In [ ]:
import pandas as pd
m = pd.read_csv('movie.csv')
r = pd.read_csv('rating.csv')

m.describe(include=['object'])

m['movieId'].info

m.columns

r.info

df = pd.merge(m, r, on='movieId', how='inner')

m.head(3)

r.head(3)

df = pd.merge(m, r, on='movieId', how='inner')

df.info()

df.drop(['title' ], axis=1, inplace = True)

df.head()

df['freq'] = df['userId'].map(df['userId'].value_counts())

df.shape

df_5 = df[df['freq'] >= 100].drop(columns=['freq'])

df_5.shape

p = df_5.pivot( index ='userId', columns='movieId', values = "rating").reset_index(drop=True)

p.head()

p.shape

p = p.iloc[0:5000,0:5000]

p.shape

p.fillna(0, inplace=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20000263 entries, 0 to 20000262
Data columns (total 6 columns):
 #   Column     Dtype  
---  ------     -----  
 0   movieId    int64  
 1   title      object 
 2   genres     object 
 3   userId     int64  
 4   rating     float64
 5   timestamp  object 
dtypes: float64(1), int64(2), object(3)
memory usage: 915.5+ MB


In [9]:

from sklearn.metrics import pairwise_distances
from scipy.spatial.distance import cosine, correlation
from sklearn.metrics.pairwise import cosine_similarity

def create_item_similarity_matrix(df_5):
    # Create a pivot table with movieId as index and userId as columns, filled with ratings
    movie_user_matrix = df_5.pivot(index='movieId', columns='userId', values='rating').fillna(0)
    
    # Compute cosine similarity between movies
    item_similarity = cosine_similarity(movie_user_matrix)
    
    # Convert to DataFrame for easier handling
    item_similarity_df = pd.DataFrame(item_similarity, index=movie_user_matrix.index, columns=movie_user_matrix.index)
    
    return item_similarity_df


In [5]:

def recommend_movies_item_based(user_id, item_similarity_df, ratings_df, movies_df, top_n=50):
    # Step 1: Get movies that the user has watched and rated highly
    user_ratings = ratings_df[ratings_df.userId == user_id]
    high_rated_movies = user_ratings[user_ratings.rating >= 4.0]['movieId']  # Assuming 4.0+ is a high rating
    
    # Step 2: Find similar movies to the ones the user liked
    recommendations = {}
    
    for movie in high_rated_movies:
        if movie in item_similarity_df.index:
            similar_movies = item_similarity_df[movie].nlargest(10).index  # Top 10 similar movies
            for sim_movie in similar_movies:
                if sim_movie not in user_ratings['movieId'].values:  # Only recommend unseen movies
                    recommendations[sim_movie] = recommendations.get(sim_movie, 0) + item_similarity_df.loc[movie, sim_movie]

    # Step 3: Sort recommendations by similarity score and get top_n recommendations
    recommended_movies = sorted(recommendations, key=recommendations.get, reverse=True)[:top_n]
    
    # Get movie titles for the recommendations
    recommended_movie_titles = movies_df[movies_df.movieId.isin(recommended_movies)]
    return recommended_movie_titles[['movieId', 'title']]

# Example usage
item_similarity_df = create_item_similarity_matrix(df_5)
recommendations = recommend_movies_item_based(user_id=5, item_similarity_df=item_similarity_df, ratings_df=r, movies_df=m, top_n=50)
print(recommendations)

NameError: name 'df_5' is not defined